<a href="https://colab.research.google.com/github/jaehyun-shin/lanchain_tutorials/blob/main/Langchain_tutorials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개발환경 세팅

In [5]:
pip install langchain langchain-openai langchain_core

# 환경변수 설정

In [13]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = userdata.get("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
LLM_MODEL = "gpt-3.5-turbo"

# 언어 모델 설정
모델을 직접 사용합니다. ChatModel은 Langchain의 "Runnables"의 인스턴스입니다.
상호작요을 위한 표준 인터페이스이지요.
모델에 메시지를 넣고 실행을 하기위한 메소드는 `.invoke` 입니다.

In [14]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model=LLM_MODEL)

# 모델 사용

## Message
LLM을 통해 대화하려면 messages를 활용해야 합니다.

`SystemMessage`의 경우 LLM의 기본 배경(`prompt`) 설정 및 LLM이 어떤 역할이고 어떠한 작업을 하는지 명시할 수 있습니다.

`HummanMessage`의 경우 유저가 직접 LLM에게 입력하는 자연어 대화입니다. 이를 통해 LLM에게 질의 할 수 있습니다.

LLM 모델의 답변은 `AIMessage`로 응답합니다.

In [19]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate to Korean"),
    HumanMessage(content="this is langchain tutorial. let's build a simple LLM Application")
]

model.invoke(messages)

AIMessage(content='이것은 Langchain 튜토리얼입니다. 간단한 LLM 애플리케이션을 만들어 봅시다.', response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 28, 'total_tokens': 72}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0ba5a431-8f6c-4985-a311-9e4d7a351d87-0', usage_metadata={'input_tokens': 28, 'output_tokens': 44, 'total_tokens': 72})

## OutputParsers
OutputParser는 모델의 응답을 구문 분석하여 반환합니다.

In [17]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)
parser.invoke(result)

'이것은 Langchain 튜토리얼입니다. 간단한 LLM 애플리케이션을 만들어 봅시다.'

## Chain
OutputParser와 Model을 연결할 수 있습니다. 즉 모델과 출력을 연결해 놓은 chain을 구동시키면 자동으로 결과가 구문 분석으로 반환된 값이 반환됩니다.

In [18]:
chain = model | parser

chain.invoke(messages)

'이것은 Langchain 튜토리얼입니다. 간단한 LLM 애플리케이션을 만들어 봅시다.'

##Prompt
메시지의 목록은 일반적으로 사용자의 입력과 애플리케이션 로직의 조합으로 구성됩니다. 미리 작성해둔

SystemMessage로 LLM의 역할을 정의하고 사용자의 입력(HumanMessage)에 대응해 답변을 내놓았습니다.

하지만 한국어로 변환하는 것이 아닌 다른 언어로 번역하고 싶은 경우 우리는 SystemMessage를 다시 손봐야합니다.

따라서 번역할 언어 부분을 변수로 빼놓아 동적으로 만들어야 합니다.

PromptTemplate은 이러한 변환을 지원하도록 설계된 Langchain의 개념입니다.

In [22]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate to {language}: "

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", "{text}")
    ]
)

result = prompt_template.invoke({"language": "korean", "text": "this is langchain tutorial. let's build a simple LLM Application"})

print(result.to_messages())

# chain에 결합

chain = prompt_template | model | parser

chain.invoke({"language": "korean", "text": "hi"})

[SystemMessage(content='Translate to korean: '), HumanMessage(content="this is langchain tutorial. let's build a simple LLM Application")]


'안녕하세요'

이로서 간단한 챗 어플리케이션 chain을 구축하였습니다.